# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
import os
import pandas
from dotenv import load_dotenv

dotenv_path = '../../01_materials/labs/.env'
load_dotenv(dotenv_path=dotenv_path)


In [63]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
import warnings
warnings.filterwarnings("ignore")

In [64]:
#import logger
import sys
sys.path.append("../../05_src")
from logger import get_logger

_logs = get_logger(__name__)

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv('PRICE_DATA') # Get path to PRICE_DATA from env file
_logs.info("Folder structure pointing to PRICE_DATA in env is : %s", PRICE_DATA)

parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet")) # Load the list of data file paths
_logs.info("Total number of parquest files in the price data at given path :  %s is : %s", PRICE_DATA, len(parquet_files))


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.

dd_px = dd.read_parquet(PRICE_DATA).set_index("ticker") # load parquet files into dask data frame
_logs.info("Columns in dask dataframe : \n %s" , dd_px.columns) # columns in dataframe

# Feature data set transformation
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))
_logs.info("\n\nDask Feature data frame columns after transformation :\n %s ",dd_feat.columns)
_logs.info("\n\nDask Feature Data set after transformation: \n\n %s ",dd_feat.head(5))

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [67]:
# Write your code below.
dd_pd = dd_feat.compute() # convert to panda data frame


In [ ]:
# Test to ensure the data frames data types
_logs.info("Data type of feature dask data frame : %s", type(dd_feat))
_logs.info("Data type of data frame after converting dask data frame in pandas : %s", type(dd_pd))

In [ ]:
# Add a rolling average return calculation with a window of 10 days
dd_pd = (dd_pd.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_avg_return = x['returns'].rolling(10).mean())
))

_logs.info(dd_pd.shape)
_logs.info('Pandas Data frame - \n\n {}'.format(dd_pd.loc["AAPL"][['Adj Close','returns', 'hi_lo_range', 'rolling_avg_return']].head(5)))

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

**Answer**

1) Dask can calculate the moving average returns and there is no real need to convert the data frame to pandas in our case.

2) Both Pandas DataFrames and Dask DataFrames have their strengths and weaknesses, making them suitable for different scenarios. In our scenario we are dealing with relatively small datasets, converting to Pandas is not a bad idea. Pandas library has a ease to use and rich functionalitu that is used in data analysis and visualization. It comes with benefit of in-memory processing which leads to fast execution for our datasets that fit within available memory. 

On the other hand, if we are working with big data and which requires efficient parallel or distributed computing, Dask DataFrames offer an impressive solution. Our choice ultimately depends on the size of our data, the complexity of our tasks, and most important willingness to explore and adopt new tools for optimal performance and scalability.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.